In [26]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import torch
import tensorflow as tf
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.manifold import TSNE
from sklearn import preprocessing 
import matplotlib.pyplot as plt
import pandas as pd 
import numpy as np
import seaborn as sns
sns.set(style="whitegrid")
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import umap
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import matplotlib.pyplot as plt
import os
for dirname, _, filenames in os.walk('./kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

./kaggle/input/train.csv
./kaggle/input/test.csv
./kaggle/input/sample_solution.csv


In [32]:
path = './kaggle/input/'
trainpath = os.path.join(path,'train.csv')
testpath = os.path.join(path,'test.csv')
samplepath = os.path.join(path,'sample_solution.csv')
X_test = pd.read_csv(testpath)

df_train = pd.read_csv(trainpath)

In [28]:
df_train.head(5)

,id,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f110,f111,f112,f113,f114,f115,f116,f117,f118,claim
0,0,0.10859,0.004314,-37.566,0.017364,0.28915,-10.25100,135.12,168900.0,3.992400e+14,...,-12.2280,1.7482,1.90960,-7.11570,4378.80,1.2096,8.613400e+14,140.1,1.01770,1
1,1,0.10090,0.299610,11822.000,0.276500,0.45970,-0.83733,1721.90,119810.0,3.874100e+15,...,-56.7580,4.1684,0.34808,4.14200,913.23,1.2464,7.575100e+15,1861.0,0.28359,0
2,2,0.17803,-0.006980,907.270,0.272140,0.45948,0.17327,2298.00,360650.0,1.224500e+13,...,-5.7688,1.2042,0.26290,8.13120,45119.00,1.1764,3.218100e+14,3838.2,0.40690,1
3,3,0.15236,0.007259,780.100,0.025179,0.51947,7.49140,112.51,259490.0,7.781400e+13,...,-34.8580,2.0694,0.79631,-16.33600,4952.40,1.1784,4.533000e+12,4889.1,0.51486,1
4,4,0.11623,0.502900,-109.150,0.297910,0.34490,-0.40932,2538.90,65332.0,1.907200e+15,...,-13.6410,1.5298,1.14640,-0.43124,3856.50,1.4830,-8.991300e+12,NaN,0.23049,1


In [29]:
vc = df_train['claim'].value_counts().to_frame().reset_index()
vc['percent'] = vc["claim"].apply(lambda x : round(100*float(x) / len(df_train), 2))
vc = vc.rename(columns = {"index" : "Target", "claim" : "Count"})
vc

,Target,Count,percent
0,0,480404,50.15
1,1,477515,49.85


In [ ]:
features = df_train.columns[:-1]
target = df_train.columns[-1]
df_train["Missing_Value"] = df_train[features].isna().sum(axis=1) 
df_train["Missing_Value_cat"] = df_train["Missing_Value"].map(lambda x: 0 if x==0 else 1)
X_test["Missing_Value"] = X_test[features].isna().sum(axis=1) 
X_test["Missing_Value_cat"] = X_test["Missing_Value"].map(lambda x: 0 if x==0 else 1)

In [ ]:
for column in features:
    avg_val = df_train[column].median()
    df_train[column].fillna(avg_val, inplace=True)
    X_test[column].fillna(avg_val, inplace=True)
features = list(features)
features.extend(['Missing_Value','Missing_Value_cat'])

In [31]:
x = df_train.drop(["claim"], axis=1)
y = df_train["claim"].values

x_scale = preprocessing.MinMaxScaler().fit_transform(x.values)
x_norm, x_claim = x_scale[y == 0], x_scale[y == 1]

(957919, 119)
(2000, 119)


In [ ]:
from keras.layers import Input, Dense
from keras.models import Model, Sequential
from keras import regularizers

In [ ]:
## input layer 
input_layer = Input(shape=(x.shape[1],))

## encoding part
encoded = Dense(512, activation='tanh', activity_regularizer=regularizers.l1(10e-5))(input_layer)
encoded = Dense(256, activation='relu')(encoded)
encoded = Dense(128, activation='relu')(encoded)
encoded = Dense(64, activation='relu')(encoded)


decoded = Dense(128, activation='tanh')(encoded)
decoded = Dense(256, activation='tanh')(decoded)
decoded = Dense(512, activation='tanh')(decoded)


## output layer
output_layer = Dense(x.shape[1], activation='relu')(decoded)

In [ ]:
autoencoder = Model(input_layer, output_layer)
autoencoder.compile(optimizer="adam", loss=tf.keras.losses.MeanSquaredLogarithmicError(
    reduction="auto", name="mean_squared_logarithmic_error"
))

In [ ]:
autoencoder.fit(x_norm, x_norm, 
                batch_size = 512, epochs = 10, 
                shuffle = True, validation_split = 0.20);

Epoch 1/10
751/751 [==============================] - 21s 27ms/step - loss: 0.0212 - val_loss: 0.0161
Epoch 2/10
751/751 [==============================] - 21s 28ms/step - loss: 0.0144 - val_loss: 0.0139
Epoch 3/10
751/751 [==============================] - 20s 27ms/step - loss: 0.0205 - val_loss: 0.0159
Epoch 4/10
751/751 [==============================] - 20s 27ms/step - loss: 0.0148 - val_loss: 0.0149
Epoch 5/10
751/751 [==============================] - 20s 27ms/step - loss: 0.0186 - val_loss: 0.0220
Epoch 6/10
751/751 [==============================] - 20s 27ms/step - loss: 0.0174 - val_loss: 0.0160
Epoch 7/10
751/751 [==============================] - 20s 27ms/step - loss: 0.0154 - val_loss: 0.0157
Epoch 8/10
751/751 [==============================] - 21s 27ms/step - loss: 0.0168 - val_loss: 0.0180
Epoch 9/10
751/751 [==============================] - 20s 27ms/step - loss: 0.0166 - val_loss: 0.0167
Epoch 10/10
751/751 [==============================] - 20s 27ms/step - loss: 0.015

In [ ]:
hidden_representation = Sequential()
n=5
for i in range(n):
    hidden_representation.add(autoencoder.layers[i])

   

In [ ]:
%%time
norm_hid_rep = hidden_representation.predict(x_norm)
claim_hid_rep = hidden_representation.predict(x_claim)
rep_x = np.append(norm_hid_rep, claim_hid_rep, axis = 0)
y_n = np.zeros(norm_hid_rep.shape[0])
y_f = np.ones(claim_hid_rep.shape[0])
rep_y = np.append(y_n, y_f)

#tried to find hidden representation, yet we could not find one...

CPU times: user 1min 24s, sys: 8.43 s, total: 1min 32s
Wall time: 42.9 s


In [ ]:
from xgboost import XGBClassifier


model = XGBClassifier(
    max_depth=3,
    subsample=0.5,
    colsample_bytree=0.5,
    n_jobs=-1,
    # Uncomment if you want to use GPU. Recommended for whole training set.
    #tree_method='gpu_hist',
    random_state=0,
)

In [ ]:
train_x, val_x, train_y, val_y = train_test_split(rep_x, rep_y, test_size=0.25)
clf = model.fit(train_x, train_y)
pred_y = clf.predict(val_x)

/opt/conda/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[06:10:49] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [ ]:
print ("")
print ("Classification Report: ")
print (classification_report(val_y, pred_y))

print ("")
print ("Accuracy Score: ", accuracy_score(val_y, pred_y))


Classification Report: 
              precision    recall  f1-score   support

         0.0       0.52      0.56      0.54    120578
         1.0       0.52      0.48      0.50    118902

    accuracy                           0.52    239480
   macro avg       0.52      0.52      0.52    239480
weighted avg       0.52      0.52      0.52    239480


Accuracy Score:  0.5177760146985134


In [ ]:
TARGET = df_train.columns[-1]
testid = X_test['id']
X_test = preprocessing.MinMaxScaler().fit_transform(X_test)

X_test = hidden_representation.predict(X_test)


In [ ]:
print(X_test)

[[nan nan nan ... nan nan nan]
 [ 0.  0.  0. ...  0.  0.  0.]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [ 0.  0.  0. ...  0.  0.  0.]
 [nan nan nan ... nan nan nan]]


In [ ]:

# Make predictions
y_pred = pd.Series(
    model.predict(X_test)
)

y_pred = pd.DataFrame(y_pred,columns=['claim'])
y_pred['id'] = testid
y_pred = y_pred[['id','claim']]


# Create submission file
y_pred.to_csv("whydiditworked.csv",index=False)



In [ ]:
from collections import Counter
y_counter = Counter(y_pred['claim'])
print(y_counter)

Counter({1.0: 354465, 0.0: 139009})
